In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
WELLCOME = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1") # The index is Date.  Add ,index_col=1 for making the date an index.
WELLCOME['COST'] = WELLCOME['COST (£) charged to Wellcome (inc VAT when charged)'] # create the "Cost" column without all the extraneous characters
WELLCOME['COST2'] = WELLCOME['COST'].apply(lambda x: x[1:]) # strip off the pound sterling sign
WELLCOME['COST2']=WELLCOME.COST2.apply(lambda x: ''.join(list(filter(str.isdigit, str(x))))).astype(float)/100 # return numeric
WELLCOME['Journal title']=WELLCOME['Journal title'].astype(str)  # set it to a string
WELLCOME['Article title']=WELLCOME['Article title'].astype(str)  # set it to a string
# WELLCOME['Journal title2']=WELLCOME['Journal title'].str.lower() # set it to lower case so that we can group by the same case

In [44]:
# Do regex replacements for the Journal Titles
JournalTitleReplacements = {
    'Acta.*Crystallographica.*D.*' : 'Acta Crystallographica Section D: Biological Crystallography',
    'Acta.*Crystallography.*D.*' : 'Acta Crystallographica Section D: Biological Crystallography',
    'Acta Neuropathol' : 'Acta Neuropathologica',
    'American Journal of Medical Genetics' : 'American Journal of Medical Genetics',
    'American Journal of Preventive Medicine' : 'American Journal of Preventive Medicine',
    'Angew' : 'Angewandte Chemie International Edition',
    'Annals of Neurology' : 'Annals of Neurology',
    'Antimi.*Chemotherapy' : 'Antimicobial Agents and Chemotherapy',
    ' \& ' : ' and ',
    'Behaviour' : 'Behavior',
    'Biinformatics' : 'Bioinformatics',
    'Biochem ' : 'Biochemical',
    'Journals' : 'Journal',
    'Bioohysica' : 'Biophysica',
    'Birth Defects Research.*Teratology.' : 'Birth Defects Research Part A: Clinical and Molecular Teratology',
    'BMC Genomics.' : 'BMC Genomics',
    'BMC Infectious [Dd]iseases' : 'BMC Infectious Diseases',
    'British Journal of Op.*' : 'British Journal of Opthalmology',
    'CHILD' : 'Child',
    'Epigentics' : 'Epigenetics',
    'Fungal Ecology' : 'Fungal Biology',
    'Epidemology' : 'Epidemiology',
    'Heptology' : 'Hepatology',
    'Hum Resour Health' : 'Human Resources for Health',
    'Joural' : 'Journal',
    'J Biol Chem.' : 'J Biol Cheminstry',
    'Journal of Acquired Immune Deficiency Syndroms (JAIDS)' : 'Journal of Acquired Immune Deficiency Syndroms',
    'therapy' : 'Therapy',
    'experiments' : 'Experiments',
    '†' : '',
    'Nephrology Dialysis Transplantation' : 'Nephrology Dialysis and Transplantation',
    'Nucleic Acid Research' : 'Nucleic Acids Research',
    'Pflugers Archiv' : 'Pflugers Archive',
    'PL.*ONE' : 'Public Library of Science ONE',
    'Proceedings of the National Academy of Sciences.*' : 'Proceedings of the National Academy of Sciences (PNAS)',
    'Public Library of Science' : 'Public Library of Science ONE',
    'Social Psychiatry and Psychiatric Epidemiol' : 'Social Psychiatry and Psychiatric Epidemiology',
    'Studies in History and Philosophy of Science Part.*' : 'Studies in History and Philosophy of Science Part C: Studies in History and Philosophy of Biological and Biomedical Sciences',
    'Trends in Neurosciences' : 'Trends in Neuroscience',
    'ACS [Nn][aA][nN][oO]' : 'ACS NANO'
}

In [48]:
# Do regex replacements for the Publishers
PublishersReplacements = {
    ' \& ' : ' and ',
    'Behaviour' : 'Behavior',
    'Biinformatics' : 'Bioinformatics',
    'Biochem ' : 'Biochemical',
    'Journals' : 'Journal',
    'Bioohysica' : 'Biophysica',
    'CHILD' : 'Child',
    'Epigentics' : 'Epigenetics',
    'Fungal Ecology' : 'Fungal Biology',
    'Epidemology' : 'Epidemiology',
    'Hamatology' : 'Hematology',
    'Haematology' : 'Hematology',
    'Heptology' : 'Hepatology',
    'Joural' : 'Journal',
    'therapy' : 'Therapy',
    'experiments' : 'Experiments',
    '†' : '',
    'ACS.*' : 'ACS',
    'Biolgy' : 'Biology',
    'ASBMB.*' : 'ASBMB/Cadmus',
    'BioMed Central.*' : 'BioMed Central',
    'BMJ.*' : 'BMJ Publishing Group',
    'CADMUS JOURNAL SERVICE' : 'Cadmus Journal Services',
    'Cambridge Uni.*' : 'Cambridge University Press',
    'Camdus Journal Services' : 'Cadmus Journal Services',
    'Cold Spring Ha.*' : 'Cold Spring Harbor Press',
    'Company of Bio.*' : 'Company of Biologists',
    'Dar.*Journal.*' : 'Dartmouth Journal Services',
    'Elsevier.*' : 'Elsevier',
    'Federation.*American.*Societ.*' : 'Federation of American Societies for Experimental Biology',
    'Frontiers Media.*' : 'Frontiers Media',
    'Future Medecine.*' : 'Future Medecine',
    'Impact Journals.*' : 'Impact Journals',
    'Informa Healthcare.*' : 'Informa Healthcare',
    'International Union of Crystallography.*' : 'International Union of Crystallography',
    'JOHN WILEY & SONS' : 'John Wiley and Sons',
    'John Wiley.*Sons.*' : 'John Wiley and Sons',
    'Landes Bioscience.*' : 'Landes Bioscience',
    'My JOVE corporation' : 'My JOVE Corporation',
    'MYJoVE Corporation' : 'My JOVE Corporation',
    'National Academy of Science.*' : 'National Academo of Sciences',
    'Nature PG' : 'Nature Publishing Group',
    'N[aA][tT][uU][rR][eE] P[uU][bB][lL][iI][sS][hH][iI][nN][gG].*' : 'Nature Publishing Group',
    'Oxford Journal.*' : 'Oxford Journal',
    'Oxford Univer.*Press.*' : 'Oxford University Press',
    'PL[Oo[sS].*]' : 'PLoS Public Library of Science',
    'PNAS.*' : 'PNAS Author Publication',
    'Royal Society.*' : 'Royal Society of Chemistry',
    'Sage Pub.*' : 'Sage Publications',
    'Society for Neuroscience.*' : 'Society for Neuroscience',
    'Springer.*' : 'Springer',
    'The American Society for Biochemistry.*' : 'The American Society for Biochemistry and Molecular Biology',
    '[tT][hH][eE] [cC][oO][mM][pP][aA][nN][yY] [oO][fF] [bB][iI][oO][lL][oO][gG].*' : 'The Company of Biologists',
    'Endrocrine' : 'Endocrine',
    'Wiley Subscription.*' : 'Wiley Subscription Services',
    'Wiley/Blackwell' : 'Wiley-Blackwell',
    'Wolters Kluwer.*' : 'Wolters Kluwer Health'
}

In [51]:
WELLCOME['Journal title']=WELLCOME['Journal title'].str.rstrip('.!? \n\t') 
WELLCOME['Publisher']=WELLCOME['Journal title'].str.rstrip('.!? \n\t') 
WELLCOME['Journal title'].replace(JournalTitleReplacements, regex=True, inplace=True)
WELLCOME['Publisher'].replace(PublishersReplacements, regex=True, inplace=True)

In [14]:
WELLCOME.dtypes

PMID/PMCID                                              object
Publisher                                               object
Journal title                                           object
Article title                                           object
COST (£) charged to Wellcome (inc VAT when charged)     object
COST                                                    object
COST2                                                  float64
dtype: object

In [32]:
WELLCOME_group_by_journal0 = WELLCOME[['Journal title']].groupby(['Journal title'])['Journal title'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)

### Top 5 journals, with article counts

In [50]:
# Top 5 journals, with article counts
WELLCOME_group_by_journal0.head(5)

,Journal title,count
754,PLoS One,92
814,Public Library of Science ONE ONE ONE ONE,87
497,Journal of Biological Chemistry,48
795,Proceedings of the National Academy of Science...,26
718,Nucleic Acids Research,24


### Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal 
Note:  There are only standard deviation calculations for journal titles with more than 1 article.

In [46]:
# Grouping by region groupings
grouping_columns = ['Journal title','COST2']
sort_by_aggregate = ['mean', 'median','count' ,'std']

WELLCOME_group_by_journal = WELLCOME.groupby(grouping_columns[0])[grouping_columns].agg(sort_by_aggregate)
WELLCOME_group_by_journal = WELLCOME_group_by_journal.sort_values(by=['Journal title'], ascending=True)

In [47]:
WELLCOME_group_by_journal.head(500)
# WELLCOME.head(20)

COST2            \
                                                             mean    median   
Journal title                                                                 
ACS Chemical Biology                                  1418.186000  1294.590   
ACS Chemical Neuroscience                             1186.800000  1186.800   
ACS NANO                                               668.140000   668.140   
ACTA F                                                 754.900000   754.900   
AGE                                                   2002.000000  2002.000   
AIDS                                                  2059.306667  1968.630   
AIDS Behav                                            1834.770000  1834.770   
AIDS Care                                             2189.170000  2189.170   
AIDS Journal                                          2015.720000  2015.720   
AIDS Research and Therapy                             1240.000000  1240.000   
AIDS UK                                               1836.920000  1836.920   
ASN Neuro                                             1453.420000  1453.420   
Academy of Nutrition and Dietetics                    2379.540000  2379.540   
Acta Crystallographica Section D: Biological Cr...     973.847143   771.420   
Acta Crystallographica Section F: Structural Bi...     796.635000   796.635   
Acta Neuropathologicaogicaogicaogicaogicaogica        1901.040000  1901.040   
Acta Neuropathologicaogicaogicaogicaogicaogicao...    2161.063333  2250.970   
Acta Opthalmologica                                   2270.160000  2270.160   
Acta Physiol                                          1991.500000  1991.500   
Addiction                                             2136.225000  2136.225   
Advances in Experimental Medicine and Biology         1928.457500  1928.460   
Age                                                   2338.280000  2338.280   
Age and Ageing                                        2040.000000  2040.000   
Ageing and Society                                    1695.000000  1695.000   
Aging Cell                                            1970.570000  1970.570   
Aids Care                                             2399.280000  2399.280   
Alcohol and Alcoholism                                2040.000000  2040.000   
Alimentrary Pharmacology and Therapeutics             2373.590000  2373.590   
Am J Bioeth                                           1804.190000  1804.190   
Am J Trop Med Hyg                                     1152.500000  1152.500   
...                                                           ...       ...   
JOURNAL OF NEUROCHEMISTRY                             2270.340000  2270.340   
Jnl Biological Chemistry                            333805.013333  1035.000   
Jnl of the International Neuropsychological Soc...    2034.000000  2034.000   
Jounral of Clinical Microbiology                      1364.470000  1364.470   
Journal Biological Chemistry                           938.750000   938.750   
Journal od Clinical Endocrinology                     3602.410000  3602.410   
Journal of Abnormal Psychology                        2534.530000  2534.530   
Journal of Acquired Immune Deficiency Syndromes       2034.750000  2034.750   
Journal of Acquired Immune Deficiency Syndroms ...    1836.920000  1836.920   
Journal of Affective Disorders                        2321.370000  2323.490   
Journal of Affective Disorders                        1530.770000  1530.770   
Journal of Aids                                       2009.650000  2009.650   
Journal of Allergy and Clinical Immunology            2148.655000  2148.655   
Journal of Alzheimer Disease                           780.250000   780.250   
Journal of Anthropological Archaeology                2470.010000  2470.010   
Journal of Antimicobial Agents and ChemoTherapy       2220.000000  2220.000   
Journal of Applied Crystallography                     758.530000   758.530   
Journal of Applied Physiology  